In [ ]:
import os
from utils import *
import utils
from agents import *
import time
import torch
import torch.nn as nn
from copy import deepcopy
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seeds',          type=int,       default=[2023, 2024, 2025])
parser.add_argument('--dataset',        type=str,       default='svhn')
parser.add_argument('--batch_size',     type=int,       default=256)
parser.add_argument('--model_name',     type=str,       default='vgg11')
parser.add_argument('--retrain',        type=bool,      default=False)
parser.add_argument('--unlearn_class',  type=list,      default=3)
args = parser.parse_args("")
args.time_str = time.strftime("%m-%d-%H-%M", time.localtime())
if args.dataset.lower() == 'fmnist':
    args.n_channels = 1
else:
    args.n_channels = 3

if args.dataset.lower() == 'cifar100':
    args.num_classes = 100
else:
    args.num_classes = 10
criterion = nn.CrossEntropyLoss()

In [3]:
def get_unlearn_dataloader(data_loader):
    dataset = data_loader.dataset
    _indices = data_loader.sampler.indices

    if args.dataset.lower() == 'svhn':
        train_targets = np.array(dataset.labels)[_indices]
    else:
        train_targets = np.array(dataset.labels)[_indices]
    unlearn_indices, remain_indices = [], []
    for i, target in enumerate(train_targets):
        if target in args.unlearn_class:
            unlearn_indices.append(i)
        else:
            remain_indices.append(i)

    unlearn_indices = np.array(_indices)[unlearn_indices]
    remain_indices = np.array(_indices)[remain_indices]

    unlearn_sampler = torch.utils.data.SubsetRandomSampler(unlearn_indices)
    unlearn_loader = torch.utils.data.DataLoader(dataset,
                                                batch_size=args.batch_size,
                                                sampler = unlearn_sampler,)

    remain_sampler = torch.utils.data.SubsetRandomSampler(remain_indices)
    remain_loader = torch.utils.data.DataLoader(dataset,
                                                batch_size=args.batch_size,
                                                sampler = remain_sampler)
    return remain_loader, unlearn_loader

def get_dataloader(args):
    train_loader, test_loader = utils.get_dataloader(args)

    indices = np.arange(len(train_loader.dataset))
    a = np.split(indices,[int(len(indices)*0.9), int(len(indices))])
    idx_train = a[0]
    idx_val = a[1]
    train_sampler = torch.utils.data.SubsetRandomSampler(idx_train)
    val_sampler = torch.utils.data.SubsetRandomSampler(idx_val)

    train_loader = torch.utils.data.DataLoader(train_loader.dataset,
                                                batch_size=args.batch_size,
                                                sampler=train_sampler)
    
    val_loader = torch.utils.data.DataLoader(train_loader.dataset,
                                                batch_size=args.batch_size,
                                                sampler=val_sampler)
    
    test_loader = torch.utils.data.DataLoader(test_loader.dataset,
                                            batch_size=args.batch_size,
                                            shuffle=False)
    
    if args.retrain:
        train_loader, _ = get_unlearn_dataloader(train_loader)
        val_loader, _ = get_unlearn_dataloader(val_loader)
        
    return train_loader, val_loader, test_loader

In [4]:
arxiv_name = 'original_model'
train_loader, val_loader, test_loader = get_dataloader(args)
train_targets_list = np.array(train_loader.dataset.labels)[train_loader.sampler.indices]
unlearn_indices = np.where(np.isin(train_targets_list, args.unlearn_class))[0]

# conver to the original indices
unlearn_indices = train_loader.sampler.indices[unlearn_indices]

unlearn_sampler = torch.utils.data.SubsetRandomSampler(unlearn_indices)
unlearn_subset_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                                    batch_size=args.batch_size, 
                                                    sampler=unlearn_sampler)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

remain_indices = np.where(~np.isin(train_targets_list, args.unlearn_class))[0]
remain_indices = train_loader.sampler.indices[remain_indices]

remain_sampler = torch.utils.data.SubsetRandomSampler(remain_indices)
remain_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                            batch_size=args.batch_size, 
                                            sampler=remain_sampler)

Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat


In [8]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{args.seeds[i]}.pth'))
        test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
0.9644, 0.9653, 0.9716, 0.9198, 0.9643, 0.9643, 0.9449, 0.9480, 0.9458, 0.9480, Acc_f: 0.9198, Acc_r: 0.9574
0.9513, 0.9723, 0.9687, 0.9410, 0.9568, 0.9379, 0.9449, 0.9297, 0.9434, 0.9386, Acc_f: 0.9410, Acc_r: 0.9493
------------ Retrained model ------------
0.9358, 0.9123, 0.9605, 0.0000, 0.9172, 0.9304, 0.9530, 0.9064, 0.8530, 0.8884, Acc_f: 0.0000, Acc_r: 0.9174
0.9713, 0.9722, 0.9687, 0.0000, 0.9738, 0.9643, 0.9565, 0.9371, 0.9301, 0.9524, Acc_f: 0.0000, Acc_r: 0.9585
0.9576, 0.9702, 0.9684, 0.0000, 0.9362, 0.9186, 0.8801, 0.8722, 0.9229, 0.9455, Acc_f: 0.0000, Acc_r: 0.9302


In [9]:
Acc_f = 100*np.array([0.9362, 0.9198, 0.9410])
Acc_r = 100*np.array([0.9549, 0.9574, 0.9493])
print(f'Original model Acc_f: {Acc_f.mean():.2f} $\pm$ {Acc_f.std():.2f}')
print(f'Original model Acc_r: {Acc_r.mean():.2f} $\pm$ {Acc_r.std():.2f}')

Acc_r = 100*np.array([0.9174, 0.9585, 0.9302])
print(f'Retrained model: {Acc_r.mean():.2f} $\pm$ {Acc_r.std():.2f}')


Original model Acc_f: 93.23 $\pm$ 0.91
Original model Acc_r: 95.39 $\pm$ 0.34
Retrained model: 93.54 $\pm$ 1.72


## Ours

## Random labeling

In [10]:
Acc_r, Acc_f = np.zeros(3), np.zeros(3)
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

    for ep in range(20):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        Acc_r[i], Acc_f[i] = test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
[train] epoch 0, batch 29, loss 4.423434734344482
0.9685, 0.9761, 0.9706, 0.7429, 0.9639, 0.9572, 0.9393, 0.9326, 0.9229, 0.9166, Acc_f: 0.7429, Acc_r: 0.9497
[train] epoch 1, batch 29, loss 2.9884297847747803
0.9604, 0.9773, 0.9585, 0.1582, 0.9358, 0.9241, 0.8862, 0.8375, 0.8349, 0.7674, Acc_f: 0.1582, Acc_r: 0.8980
[train] epoch 2, batch 29, loss 2.679971933364868
0.9547, 0.9737, 0.9450, 0.0392, 0.9267, 0.8972, 0.8467, 0.7568, 0.7645, 0.6903, Acc_f: 0.0392, Acc_r: 0.8617
[train] epoch 3, batch 29, loss 2.4780831336975098
0.9518, 0.9686, 0.9356, 0.0139, 0.9187, 0.8888, 0.8265, 0.7023, 0.7151, 0.6276, Acc_f: 0.0139, Acc_r: 0.8372
[train] epoch 4, batch 29, loss 2.443204164505005
0.9490, 0.9639, 0.9262, 0.0059, 0.9152, 0.8872, 0.8159, 0.6647, 0.6783, 0.5824, Acc_f: 0.0059, Acc_r: 0.8203
[train] epoch 5, batch 29, loss 2.341336250305176
0.9472, 0.9594, 0.9159, 0.0028, 0.9045, 0.89

In [12]:
Acc_f = 100*np.array([0.0139, 0.0111, 0.0101])
Acc_r = 100*np.array([0.8372, 0.8010, 0.6379])
print(f'Random label Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Random label Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Random label Acc_f: 1.17 \pm 0.16
Random label Acc_r: 75.87 \pm 8.67


In [11]:
Proj_mat_lst =[]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    test_by_class(model, test_loader, i=args.unlearn_class)
    
    feature_list = []
    merged_feat_mat = []
    for batch, (x, y) in enumerate(remain_loader):
        x = x.cuda()
        y = y.cuda()
        mat_list = get_representation_matrix(model, x, batch_list=[256]*30)
        break
    threshold = 0.99
    merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
    proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
    Proj_mat_lst.append(proj_mat)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
Threshold:  0.99
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 62/576
Layer 3 : 734/1152
Layer 4 : 1835/2304
Layer 5 : 844/2304
Layer 6 : 896/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 29/512
Layer 10 : 9/4096
Layer 11 : 8/4096
----------------------------------------
0.9644, 0.9653, 0.9716, 0.9198, 0.9643, 0.9643, 0.9449, 0.9480, 0.9458, 0.9480, Acc_f: 0.9198, Acc_r: 0.9574
Threshold:  0.99
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 60/576
Layer 3 : 739/1152
Layer 4 : 1832/2304
Layer 5 : 843/2304
Layer 6 : 899/4608
Layer 7 : 0/4608
Layer 8 : 0/4608
Layer 9 : 27/512
Layer 10 : 9/4096
Layer 11 : 8/4096
----------------------------------------
0.9513, 0.9723, 0.9687, 0.9410, 0.9568, 0.9379, 0.94

In [16]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
[train] epoch 0, batch 29, loss 2.62646484375
0.9667, 0.9694, 0.9689, 0.0906, 0.9734, 0.9560, 0.9509, 0.9366, 0.9289, 0.9310, Acc_f: 0.0906, Acc_r: 0.9535
[train] epoch 1, batch 29, loss 2.4480502605438232
0.9662, 0.9686, 0.9679, 0.0160, 0.9738, 0.9585, 0.9555, 0.9341, 0.9247, 0.9191, Acc_f: 0.0160, Acc_r: 0.9521
[train] epoch 2, batch 29, loss 2.3905811309814453
0.9662, 0.9674, 0.9689, 0.0045, 0.9734, 0.9581, 0.9560, 0.9326, 0.9241, 0.9047, Acc_f: 0.0045, Acc_r: 0.9502
[train] epoch 3, batch 29, loss 2.260897636413574
0.9667, 0.9667, 0.9687, 0.0010, 0.9742, 0.9597, 0.9560, 0.9312, 0.9223, 0.8878, Acc_f: 0.0010, Acc_r: 0.9481
[train] epoch 4, batch 29, loss 2.329035520553589
0.9667, 0.9653, 0.9646, 0.0003, 0.9738, 0.9585, 0.9550, 0.9247, 0.9217, 0.8683, Acc_f: 0.0003, Acc_r: 0.9443
[train] epoch 5, batch 29, loss 2.2898318767547607
0.9667, 0.9647, 0.9614, 0.0000, 0.9734, 0.9610,

In [20]:
Acc_r = 100*np.array([0.9521, 0.9499, 0.9363])
Acc_f = 100*np.array([0.0160, 0.0139, 0.0566])

print(f'Random label + Subspace Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Random label + Subspace Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Random label + Subspace Acc_f: 2.88 \pm 1.97
Random label + Subspace Acc_r: 94.61 \pm 0.70


In [17]:
def get_2nd_score(model, x, y):
    indices = torch.topk(model(x), k=2, dim=1).indices
    top1_matches = indices[:, 0] == y
    selected_labels = torch.where(top1_matches, indices[:, 1], indices[:, 0])
    return selected_labels

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.03)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = get_2nd_score(model, x, y.cuda())
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
[train] epoch 0, batch 29, loss 0.5311841368675232
0.9673, 0.9739, 0.9788, 0.0000, 0.9620, 0.9673, 0.9535, 0.9386, 0.9464, 0.9335, Acc_f: 0.0000, Acc_r: 0.9579
[train] epoch 1, batch 29, loss 0.47979700565338135
0.9644, 0.9692, 0.9773, 0.0000, 0.9663, 0.9706, 0.9525, 0.9445, 0.9530, 0.9304, Acc_f: 0.0000, Acc_r: 0.9587
[train] epoch 2, batch 29, loss 0.2808799147605896
0.9667, 0.9720, 0.9761, 0.0000, 0.9671, 0.9631, 0.9595, 0.9416, 0.9524, 0.9285, Acc_f: 0.0000, Acc_r: 0.9586
[train] epoch 3, batch 29, loss 0.2709907293319702
0.9667, 0.9714, 0.9769, 0.0000, 0.9631, 0.9669, 0.9509, 0.9445, 0.9584, 0.9335, Acc_f: 0.0000, Acc_r: 0.9592
[train] epoch 4, batch 29, loss 0.523321807384491
0.9690, 0.9741, 0.9771, 0.0000, 0.9627, 0.9555, 0.9540, 0.9450, 0.9602, 0.9348, Acc_f: 0.0000, Acc_r: 0.9592
[train] epoch 5, batch 29, loss 0.3216308653354645
0.9656, 0.9735, 0.9696, 0.0000, 0.9655, 

In [ ]:
Acc_r = 100*np.array([0.9597, 0.9613, 0.9520])
Acc_f = 100*np.array([0.0000, 0.0000, 0.0000])
print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

# Boundary Unlearning

In [13]:
from agents.adv import FGSM

def find_adjacent_cls(adv_agent, x, y):
    x_adv = adv_agent.perturb(x, y)
    adv_logits = model(x_adv)
    adv_pred = torch.argmax(adv_logits.data, 1)
    return adv_pred, x_adv

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))  

    adv_agent = FGSM(deepcopy(model), bound=0.5, norm=False, random_start=True, device='cuda')
    sgd_mr_model = deepcopy(model)
    print('==='*60)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.001)

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            adv_pred, x_adv = find_adjacent_cls(adv_agent, x, y)
            adv_y = torch.argmax(model(x_adv), dim=1).detach().cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, adv_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549
[train] epoch 0, batch 29, loss 2.1821048259735107
0.9157, 0.9831, 0.9489, 0.0219, 0.8692, 0.9220, 0.7886, 0.6612, 0.7837, 0.3411, Acc_f: 0.0219, Acc_r: 0.8015
[train] epoch 1, batch 29, loss 2.0913467407226562
0.8939, 0.9886, 0.9197, 0.0330, 0.8292, 0.9249, 0.7845, 0.5656, 0.8018, 0.1824, Acc_f: 0.0330, Acc_r: 0.7656
[train] epoch 2, batch 29, loss 2.014497756958008
0.8647, 0.9943, 0.8318, 0.0402, 0.7634, 0.8813, 0.7527, 0.4482, 0.8018, 0.1034, Acc_f: 0.0402, Acc_r: 0.7157
[train] epoch 3, batch 29, loss 1.945489764213562
0.8544, 0.9953, 0.8110, 0.0854, 0.7380, 0.8616, 0.7405, 0.4106, 0.8060, 0.0740, Acc_f: 0.0854, Acc_r: 0.6990
[train] epoch 4, batch 29, loss 2.079373598098755


0.8400, 0.9953, 0.8286, 0.1249, 0.7198, 0.8779, 0.7157, 0.3829, 0.8084, 0.0589, Acc_f: 0.1249, Acc_r: 0.6920
[train] epoch 5, batch 29, loss 1.9520225524902344
0.8389, 0.9953, 0.8467, 0.1634, 0.7122, 0.8624, 0.7051, 0.3824, 0.8151, 0.0564, Acc_f: 0.1634, Acc_r: 0.6905
[train] epoch 6, batch 29, loss 1.9265741109848022
0.8406, 0.9953, 0.8501, 0.1728, 0.7095, 0.8523, 0.7076, 0.3834, 0.8343, 0.0552, Acc_f: 0.1728, Acc_r: 0.6920
[train] epoch 7, batch 29, loss 1.9234846830368042
0.8303, 0.9953, 0.8450, 0.1308, 0.6770, 0.8440, 0.6930, 0.3655, 0.8271, 0.0495, Acc_f: 0.1308, Acc_r: 0.6807
[train] epoch 8, batch 29, loss 1.900150179862976
0.8446, 0.9947, 0.8670, 0.2474, 0.6996, 0.8561, 0.7233, 0.3868, 0.8488, 0.0596, Acc_f: 0.2474, Acc_r: 0.6978
[train] epoch 9, batch 29, loss 1.9177441596984863
0.8498, 0.9947, 0.8785, 0.2578, 0.7079, 0.8708, 0.7360, 0.4156, 0.8620, 0.0633, Acc_f: 0.2578, Acc_r: 0.7087
[train] epoch 10, batch 29, loss 1.8469661474227905
0.8498, 0.9949, 0.8766, 0.2217, 0.7071, 

In [20]:
from agents.adv import FGSM

def find_adjacent_cls(adv_agent, x, y):
    x_adv = adv_agent.perturb(x, y)
    adv_logits = model(x_adv)
    adv_pred = torch.argmax(adv_logits.data, 1)
    return adv_pred, x_adv

for i in range(2, 3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))  

    adv_agent = FGSM(deepcopy(model), bound=0.5, norm=False, random_start=True, device='cuda')
    sgd_mr_model = deepcopy(model)
    print('==='*60)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.001)

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            adv_pred, x_adv = find_adjacent_cls(adv_agent, x, y)
            adv_y = torch.argmax(model(x_adv), dim=1).detach().cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, adv_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9513, 0.9723, 0.9687, 0.9410, 0.9568, 0.9379, 0.9449, 0.9297, 0.9434, 0.9386, Acc_f: 0.9410, Acc_r: 0.9493
[train] epoch 0, batch 29, loss 2.134913921356201
0.6325, 0.9645, 0.5924, 0.2065, 0.7063, 0.9723, 0.7274, 0.3725, 0.6199, 0.1498, Acc_f: 0.2065, Acc_r: 0.6375
[train] epoch 1, batch 29, loss 2.041485071182251
0.5476, 0.9873, 0.5577, 0.2738, 0.6595, 0.9354, 0.6935, 0.2744, 0.6614, 0.1135, Acc_f: 0.2738, Acc_r: 0.6034
[train] epoch 2, batch 29, loss 1.991804599761963
0.5126, 0.9890, 0.5421, 0.3123, 0.6377, 0.9299, 0.6748, 0.2382, 0.6849, 0.0903, Acc_f: 0.3123, Acc_r: 0.5888
[train] epoch 3, batch 29, loss 1.968560814857483
0.5178, 0.9910, 0.5401, 0.2890, 0.6457, 0.9195, 0.6884, 0.2402, 0.7120, 0.1034, Acc_f: 0.2890, Acc_r: 0.5953
[train] epoch 4, batch 29, loss 1.9913945198059082


0.5487, 0.9914, 0.6035, 0.2856, 0.6829, 0.9123, 0.6970, 0.2759, 0.7337, 0.1417, Acc_f: 0.2856, Acc_r: 0.6208
[train] epoch 5, batch 29, loss 1.9811981916427612
0.5677, 0.9918, 0.6226, 0.2720, 0.6932, 0.9102, 0.7248, 0.2942, 0.7651, 0.1661, Acc_f: 0.2720, Acc_r: 0.6373
[train] epoch 6, batch 29, loss 1.9128860235214233
0.5786, 0.9912, 0.6775, 0.2689, 0.7158, 0.9195, 0.7350, 0.3289, 0.7801, 0.1812, Acc_f: 0.2689, Acc_r: 0.6564
[train] epoch 7, batch 29, loss 1.8777530193328857
0.5958, 0.9924, 0.6763, 0.2644, 0.7582, 0.9119, 0.7516, 0.3556, 0.8006, 0.1925, Acc_f: 0.2644, Acc_r: 0.6705
[train] epoch 8, batch 29, loss 1.842572569847107
0.6365, 0.9908, 0.7344, 0.3591, 0.7772, 0.9203, 0.7683, 0.4131, 0.8169, 0.2351, Acc_f: 0.3591, Acc_r: 0.6992
[train] epoch 9, batch 29, loss 1.9199988842010498
0.6330, 0.9922, 0.7390, 0.2974, 0.7685, 0.9048, 0.7719, 0.4289, 0.8355, 0.2677, Acc_f: 0.2974, Acc_r: 0.7046
[train] epoch 10, batch 29, loss 1.8599315881729126
0.6514, 0.9886, 0.7816, 0.3439, 0.7883, 

In [22]:
Acc_r = 100*np.array([0.8015, 0.8627,  0.7656])
Acc_f = 100*np.array([0.0219, 0.0659,  0.0330])
print(f'Boundary Unlearning Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Boundary Unlearning Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Boundary Unlearning Acc_f: 4.03 \pm 1.87
Boundary Unlearning Acc_r: 80.99 \pm 4.01


# Gradient Ascent

In [23]:
for i in range(3):
    print('\n\n')
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))  

    sgd_mr_model = deepcopy(model)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.00008)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

    for ep in range(30):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = y.cuda()
            pred_y = sgd_mr_model(x)
            loss = -criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)




[train] epoch 0, batch 29, loss -0.09847826510667801
0.9662, 0.9678, 0.9670, 0.9334, 0.9683, 0.9513, 0.9474, 0.9470, 0.9434, 0.9404, Acc_f: 0.9334, Acc_r: 0.9554
[train] epoch 1, batch 29, loss -0.08678672462701797
0.9662, 0.9680, 0.9677, 0.9303, 0.9687, 0.9526, 0.9479, 0.9475, 0.9452, 0.9411, Acc_f: 0.9303, Acc_r: 0.9561
[train] epoch 2, batch 29, loss -0.03984031826257706
0.9667, 0.9684, 0.9682, 0.9251, 0.9687, 0.9547, 0.9484, 0.9485, 0.9458, 0.9404, Acc_f: 0.9251, Acc_r: 0.9567
[train] epoch 3, batch 29, loss -0.11497019976377487
0.9667, 0.9686, 0.9684, 0.9198, 0.9687, 0.9568, 0.9494, 0.9490, 0.9446, 0.9411, Acc_f: 0.9198, Acc_r: 0.9570
[train] epoch 4, batch 29, loss -0.08632414042949677


0.9667, 0.9686, 0.9691, 0.9129, 0.9691, 0.9581, 0.9494, 0.9490, 0.9446, 0.9411, Acc_f: 0.9129, Acc_r: 0.9573
[train] epoch 5, batch 29, loss -0.15785294771194458
0.9667, 0.9690, 0.9696, 0.9011, 0.9703, 0.9597, 0.9494, 0.9495, 0.9452, 0.9411, Acc_f: 0.9011, Acc_r: 0.9578
[train] epoch 6, batch 29, loss -0.16154815256595612
0.9667, 0.9694, 0.9694, 0.8938, 0.9699, 0.9618, 0.9509, 0.9495, 0.9464, 0.9417, Acc_f: 0.8938, Acc_r: 0.9584
[train] epoch 7, batch 29, loss -0.16644635796546936
0.9667, 0.9692, 0.9694, 0.8848, 0.9703, 0.9639, 0.9504, 0.9490, 0.9470, 0.9423, Acc_f: 0.8848, Acc_r: 0.9587
[train] epoch 8, batch 29, loss -0.12518297135829926
0.9667, 0.9692, 0.9704, 0.8730, 0.9707, 0.9648, 0.9504, 0.9500, 0.9464, 0.9423, Acc_f: 0.8730, Acc_r: 0.9590
[train] epoch 9, batch 29, loss -0.15574957430362701
0.9667, 0.9696, 0.9708, 0.8543, 0.9707, 0.9648, 0.9504, 0.9500, 0.9482, 0.9429, Acc_f: 0.8543, Acc_r: 0.9594
[train] epoch 10, batch 29, loss -0.38853782415390015
0.9667, 0.9700, 0.9701, 0.8

In [24]:
Acc_r = 100*np.array([0.7419,  0.6733,  0.8381])
Acc_f = 100*np.array([0.0070,  0.0000,  0.0861])

print(f'GA Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'GA Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

GA Acc_f: 3.10 \pm 3.90
GA Acc_r: 75.11 \pm 6.76


## Fisher unlearning

In [25]:
import copy
import torch.nn.functional as F

def hessian(dataset, model):
    model.eval()
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
    loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
    device = torch.device("cuda")

    for p in model.parameters():
        p.grad2_acc = 0
    
    for data, orig_target in tqdm(train_loader):
        data, orig_target = data.to(device), orig_target.to(device)
        output = model(data)
        prob = F.softmax(output, dim=-1).data

        for y in range(output.shape[1]):
            target = torch.empty_like(orig_target).fill_(y)
            loss = loss_fn(output, target)
            model.zero_grad()
            loss.backward(retain_graph=True)
            for p in model.parameters():
                if p.requires_grad:
                    p.grad2_acc += torch.mean(prob[:, y]) * p.grad.data.pow(2) 

    for p in model.parameters():
        p.grad2_acc /= len(train_loader)
    
def get_mean_var(args, p, alpha=1e-7):
    var = copy.deepcopy(1./(p.grad2_acc+1e-8))
    var = var.clamp(max=1e3) 
    if p.size(0) == args.num_classes:
        var = var.clamp(max=1e2)
    var = alpha * var 
    
    if p.ndim > 1:
        var = var.mean(dim=1, keepdim=True).expand_as(p).clone()
    mu = copy.deepcopy(p.data0.clone())

    if p.size(0) == args.num_classes:
        mu[args.unlearn_class] = 0
        var[args.unlearn_class] = 0.0001
        var *= 10
    elif p.ndim == 1:
        var *= 10 
    return mu, var

def fisher_new(dataset, model):
    for p in model.parameters():
        p.data0 = copy.deepcopy(p.data.clone())
    hessian(dataset, model)
    for i, p in enumerate(model.parameters()):
        mu, var = get_mean_var(args, p)
        p.data = mu + var.sqrt() * torch.empty_like(p.data).normal_()
    return model

In [29]:

remain_dataset = torch.utils.data.Subset(train_loader.dataset, remain_indices)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))  
    test_by_class(model, test_loader, i=args.unlearn_class)
    fisher_model = copy.deepcopy(model)
    fisher_new(remain_dataset, fisher_model)
    test_by_class(fisher_model, test_loader, i=args.unlearn_class)

0.9656, 0.9674, 0.9663, 0.9362, 0.9683, 0.9492, 0.9469, 0.9470, 0.9434, 0.9398, Acc_f: 0.9362, Acc_r: 0.9549


 51%|█████     | 58/114 [00:09<00:09,  6.20it/s]

100%|██████████| 114/114 [00:18<00:00,  6.21it/s]


0.9644, 0.9676, 0.9745, 0.0003, 0.9699, 0.9652, 0.9550, 0.9485, 0.9476, 0.9367, Acc_f: 0.0003, Acc_r: 0.9588
0.9644, 0.9653, 0.9716, 0.9198, 0.9643, 0.9643, 0.9449, 0.9480, 0.9458, 0.9480, Acc_f: 0.9198, Acc_r: 0.9574


100%|██████████| 114/114 [00:18<00:00,  6.20it/s]


0.9667, 0.9718, 0.9759, 0.0413, 0.9675, 0.9727, 0.9489, 0.9411, 0.9482, 0.9524, Acc_f: 0.0413, Acc_r: 0.9606
0.9513, 0.9723, 0.9687, 0.9410, 0.9568, 0.9379, 0.9449, 0.9297, 0.9434, 0.9386, Acc_f: 0.9410, Acc_r: 0.9493


100%|██████████| 114/114 [00:18<00:00,  6.20it/s]


0.9536, 0.9767, 0.9773, 0.0805, 0.9584, 0.9572, 0.9459, 0.9217, 0.9380, 0.9373, Acc_f: 0.0805, Acc_r: 0.9518


In [27]:
Acc_r = 100*np.array([0.9596,  0.9590, 0.9530])
Acc_f = 100*np.array([0.0378,  0.0163, 0.0219])

print(f'Fisher Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Fisher Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Fisher Acc_f: 2.53 \pm 0.91
Fisher Acc_r: 95.72 \pm 0.30


In [11]:
import copy
import torch.nn.functional as F

def hessian(dataset, model):
    model.eval()
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
    loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
    device = torch.device("cuda")

    for p in model.parameters():
        p.grad2_acc = 0
    
    for data, orig_target in tqdm(train_loader):
        data, orig_target = data.to(device), orig_target.to(device)
        output = model(data)
        prob = F.softmax(output, dim=-1).data

        for y in range(output.shape[1]):
            target = torch.empty_like(orig_target).fill_(y)
            loss = loss_fn(output, target)
            model.zero_grad()
            loss.backward(retain_graph=True)
            for p in model.parameters():
                if p.requires_grad:
                    p.grad2_acc += torch.mean(prob[:, y]) * p.grad.data.pow(2) 

    for p in model.parameters():
        p.grad2_acc /= len(train_loader)
    
def get_mean_var(args, p, alpha=1.25e-7):
    var = copy.deepcopy(1./(p.grad2_acc+1e-8))
    var = var.clamp(max=1e3) 
    if p.size(0) == args.num_classes:
        var = var.clamp(max=1e2)
    var = alpha * var 
    
    if p.ndim > 1:
        var = var.mean(dim=1, keepdim=True).expand_as(p).clone()
    mu = copy.deepcopy(p.data0.clone())

    if p.size(0) == args.num_classes:
        mu[unlearn_class] = 0
        var[unlearn_class] = 0.0001
        var *= 10
    elif p.ndim == 1:
        var *= 10 
    return mu, var

def fisher_new(dataset, model):
    for p in model.parameters():
        p.data0 = copy.deepcopy(p.data.clone())
    hessian(dataset, model)
    for i, p in enumerate(model.parameters()):
        mu, var = get_mean_var(args, p)
        p.data = mu + var.sqrt() * torch.empty_like(p.data).normal_()
    return model

In [5]:
unlearn_class = 6
remain_class = list(set(list(range(10))) -set([unlearn_class]))
train_targets_list = np.array(train_loader.dataset.labels)
remain_cls_indices = np.where(~np.isin(train_targets_list, unlearn_class))[0]
cls_sampler = torch.utils.data.SubsetRandomSampler(remain_cls_indices)
remain_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                            batch_size=args.batch_size, 
                                            sampler=cls_sampler)

In [22]:
fisher_model = copy.deepcopy(model)
criterion = torch.nn.CrossEntropyLoss()
remain_dataset = torch.utils.data.Subset(train_loader.dataset, remain_cls_indices)
fisher_new(remain_dataset, fisher_model)
test_by_class(fisher_model, test_loader, i=unlearn_class)

  0%|          | 7/1688 [00:00<00:24, 68.84it/s]

100%|██████████| 1688/1688 [00:20<00:00, 83.24it/s]


0.8660, 0.1360, 0.7570, 0.7100, 0.9090, 0.9630, 0.0620, 0.3850, 0.9250, 0.9830, mean_acc: 0.7371


In [23]:
train_loader, test_loader = get_dataloader(args)
remain_dataset = torch.utils.data.Subset(train_loader.dataset, remain_cls_indices)

for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    test_by_class(model, test_loader, i=6)
    fisher_model = copy.deepcopy(model)
    fisher_new(remain_dataset, fisher_model)
    test_by_class(fisher_model, test_loader, i=unlearn_class)

0.8580, 0.9840, 0.9130, 0.9510, 0.8320, 0.9740, 0.7910, 0.9820, 0.9840, 0.9650, mean_acc: 0.9381


100%|██████████| 1688/1688 [00:22<00:00, 73.73it/s]


0.8380, 0.5230, 0.9720, 0.8160, 0.7210, 0.8370, 0.0020, 0.8360, 0.9460, 0.9870, mean_acc: 0.8307
0.8740, 0.9850, 0.8920, 0.9380, 0.8150, 0.9700, 0.8120, 0.9810, 0.9850, 0.9680, mean_acc: 0.9342


100%|██████████| 1688/1688 [00:22<00:00, 75.64it/s]


0.9110, 0.1970, 0.7550, 0.8350, 0.9700, 0.9510, 0.0290, 0.9380, 0.9000, 0.9560, mean_acc: 0.8237
0.8180, 0.9840, 0.9150, 0.9330, 0.8350, 0.9630, 0.8460, 0.9640, 0.9870, 0.9780, mean_acc: 0.9308


100%|██████████| 1688/1688 [00:20<00:00, 82.07it/s]


0.9900, 0.8760, 0.6560, 0.8230, 0.8000, 0.4780, 0.0000, 0.1040, 0.7990, 0.8450, mean_acc: 0.7079


In [28]:
Acc_r = 100*np.array([0.8307, 0.8237, 0.7079])
Acc_f = 100*np.array([0.0020, 0.0290, 0.0000])

print(f'Remain {np.mean(Acc_r):.4f}-{np.std(Acc_r):.4f}')
print(f'Forget {np.mean(Acc_f):.4f}-{np.std(Acc_f):.4f}')

Remain 78.7433-5.6311
Forget 1.0333-1.3225


# SalUn

In [5]:
arxiv_name = 'original_model_12-19-21-49'
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)

remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Using downloaded and verified file: ../data/svhn/train_32x32.mat


Using downloaded and verified file: ../data/svhn/test_32x32.mat


In [6]:
# create saliency map
def save_gradient_ratio(unlearn_train_loader, model, criterion, args, seed):
    optimizer = torch.optim.SGD(
        model.parameters(),
        args.unlearn_lr,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )
    gradients = {}
    model.eval()
    for name, param in model.named_parameters():
        gradients[name] = 0

    for i, (image, target) in enumerate(unlearn_train_loader):
        image = image.cuda()
        target = target.cuda()

        # compute output
        output_clean = model(image)
        loss = - criterion(output_clean, target)

        optimizer.zero_grad()
        loss.backward()

        with torch.no_grad():
            for name, param in model.named_parameters():
                if param.grad is not None:
                    gradients[name] += param.grad.data

    with torch.no_grad():
        for name in gradients:
            gradients[name] = torch.abs_(gradients[name])

    threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    for i in threshold_list:
        print(i)
        sorted_dict_positions = {}
        hard_dict = {}

        # Concatenate all tensors into a single tensor
        all_elements = - torch.cat([tensor.flatten() for tensor in gradients.values()])

        # Calculate the threshold index for the top 10% elements
        threshold_index = int(len(all_elements) * i)

        # Calculate positions of all elements
        positions = torch.argsort(all_elements)
        ranks = torch.argsort(positions)

        start_index = 0
        for key, tensor in gradients.items():
            num_elements = tensor.numel()
            # tensor_positions = positions[start_index: start_index + num_elements]
            tensor_ranks = ranks[start_index : start_index + num_elements]

            sorted_positions = tensor_ranks.reshape(tensor.shape)
            sorted_dict_positions[key] = sorted_positions

            # Set the corresponding elements to 1
            threshold_tensor = torch.zeros_like(tensor_ranks)
            threshold_tensor[tensor_ranks < threshold_index] = 1
            threshold_tensor = threshold_tensor.reshape(tensor.shape)
            hard_dict[key] = threshold_tensor
            start_index += num_elements

        torch.save(hard_dict, f'./save/{args.dataset}/{args.model_name}/mask_threshold_{seed}_{i}.pt')


args.unlearn_lr=0.01
args.momentum=0.9
args.weight_decay=5e-4
criterion = nn.CrossEntropyLoss()

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    save_gradient_ratio(unlearn_train_loader, model, criterion, args, args.seeds[i])

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [8]:
from agents.svc_mia import SVC_MIA
indice = remain_train_loader.sampler.indices
neg_size = len(test_loader.sampler) + len(val_loader.sampler.indices)
balanced_indice = np.random.choice(indice, size=neg_size, replace=False)
balanced_sampler = torch.utils.data.SubsetRandomSampler(balanced_indice)
balanced_train_loader = torch.utils.data.DataLoader(remain_train_loader.dataset,
                                                    batch_size=args.batch_size,
                                                    sampler=balanced_sampler)

threshold = 0.8
MIA_acc = np.zeros(3)
for i in range(3):
    print("======="*50)
    mask = torch.load(f'./save/{args.dataset}/{args.model_name}/mask_threshold_{args.seeds[i]}_{threshold}.pt')
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    for ep in range(5):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            for name, param in sgd_mr_model.named_parameters():
                if param.grad is not None:
                    param.grad *= mask[name]
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

    MIA_acc[i] =  SVC_MIA(shadow_train=balanced_train_loader, 
            target_train=None, 
            target_test=unlearn_train_loader,
            shadow_test=test_loader, 
            model=sgd_mr_model)
print(f'MIA acc: {MIA_acc.mean():.2f} \pm {MIA_acc.std():.2f}')

0.9719, 0.9712, 0.9624, 0.9167, 0.9687, 0.9488, 0.9474, 0.9534, 0.9253, 0.9611, Acc_f: 0.9167, Acc_r: 0.9567
[train] epoch 0, batch 29, loss 3.429487705230713
0.9725, 0.9847, 0.9369, 0.5965, 0.9564, 0.9442, 0.9155, 0.8524, 0.8590, 0.9398, Acc_f: 0.5965, Acc_r: 0.9290
[train] epoch 1, batch 29, loss 2.7819137573242188
0.9644, 0.9876, 0.8224, 0.1235, 0.9465, 0.8612, 0.8209, 0.6652, 0.7042, 0.7574, Acc_f: 0.1235, Acc_r: 0.8366
[train] epoch 2, batch 29, loss 2.459082841873169
0.9587, 0.9880, 0.7510, 0.0416, 0.9405, 0.8112, 0.7699, 0.5760, 0.5970, 0.6502, Acc_f: 0.0416, Acc_r: 0.7825
[train] epoch 3, batch 29, loss 2.4924559593200684
0.9530, 0.9865, 0.7069, 0.0187, 0.9338, 0.7945, 0.7375, 0.5280, 0.5398, 0.5893, Acc_f: 0.0187, Acc_r: 0.7521
[train] epoch 4, batch 29, loss 2.3723368644714355
0.9518, 0.9867, 0.6855, 0.0090, 0.9255, 0.7961, 0.7117, 0.4963, 0.5078, 0.5310, Acc_f: 0.0090, Acc_r: 0.7325
0.9639, 0.9722, 0.9658, 0.9129, 0.9691, 0.9480, 0.9646, 0.9287, 0.9253, 0.9473, Acc_f: 0.9129

In [9]:
Acc_r = 100*np.array([0.7825,  0.8501,  0.7317])
Acc_f = 100*np.array([0.0416,  0.0281,  0.0371])

print(f'SalUn Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'SalUn Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

SalUn Acc_f: 3.56 \pm 0.56
SalUn Acc_r: 78.81 \pm 4.85
